# Tugas 2

- Silahkan mencari dataset lain. Dari dataset tersebut, ambil 1000 baris saja untuk mempercepat komputasi
- Dengan menggunakan kombinasi antara BOW atau TF-IDF, pasangkan dengan model klasifikasi, seperti MLP, DT, RF, LR, AdaBoost, dll
- Tampilkan hasilnya, dan pilih kombinasi terbaik (feature extraction method + classifier)

sentiment analysis using https://www.kaggle.com/datasets/anggapurnama/twitter-dataset-ppkm dataset from Kaggle and build classification models.

Label: 0 (Positive), 1 (Neutral), 2 (Negative)

In [1]:
%pip install --upgrade git+https://github.com/ariaghora/mpstemmer.git

  Cloning https://github.com/ariaghora/mpstemmer.git to /tmp/pip-req-build-_0nkcnu2
  Running command git clone --filter=blob:none --quiet https://github.com/ariaghora/mpstemmer.git /tmp/pip-req-build-_0nkcnu2
  Resolved https://github.com/ariaghora/mpstemmer.git to commit 25a5fd923af163a7eac3a5ec976984156ca8fa8b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpstemmer: filename=mpstemmer-0.1.0-py3-none-any.whl size=99801 sha256=341bcff942b19a6552ac31f2834fa4d0a158bc42d6c089c85ee5913d9691371c
  Stored in directory: /tmp/pip-ephem-wheel-cache-s9imrw1k/wheels/85/1f/78/fccc1e7e90613593e0a66304a3711d678e3013034d91534182
Successfully built mpstemmer
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

df = pd.read_csv("data/indoppkmtweet.csv", sep="\t")
df.head()

,Date,User,Tweet,sentiment
0,2022-03-31 14:32:04+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
1,2022-03-31 09:26:00+00:00,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1
2,2022-03-31 05:02:34+00:00,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1
3,2022-03-30 14:23:10+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
4,2022-03-30 11:28:57+00:00,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from collections import Counter
import string
import re
from mpstemmer import MPStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'Levenshtein'